In [2]:
import pennylane as qml
import matplotlib.pyplot as plt
from pennylane import numpy as np
from scipy.linalg import expm

from math import pi
import numpy as np
import pandas as pd
import random
import torch
from copy import deepcopy

In [16]:
n_vals = 5

dev = qml.device('default.qubit', wires = n_vals)

@qml.qnode(dev)
def rotation_circuit(vals, theta):

    # Apply Hadamards
    for hadamard_wire in range(len(vals)):
        qml.Hadamard(wires = hadamard_wire)

    # Apply value dependent Z-axis rotations
    for rotation_val in range(len(vals)):
        qml.RZ(vals[rotation_val], wires = rotation_val)

    # Random CNOT
    qml.CNOT(wires = [1,0])

    # Parametized rotation <- this is what is being trained
    qml.RY(theta, wires=0)

    # Get expected values & return them
    expected_values = [qml.expval(qml.PauliX(wire)) for wire in range(len(vals))]
    return expected_values


rotation_circuit([.14, .80, .92, .34, .1], .2)

tensor([0.9704776 , 0.68989013, 0.60582016, 0.94275467, 0.99500417], requires_grad=True)

In [187]:
# class QuantumLayer():

#     def __init__(self, n_vals):

#         self.devX = qml.device('default.qubit', wires = n_vals)
#         self.devY = qml.device('default.qubit', wires = n_vals)
#         self.devZ = qml.device('default.qubit', wires = n_vals)

#         @qml.qnode(self.devX)
#         def rotation_circuitX(vals, theta):

#             # Apply Hadamards
#             for hadamard_wire in range(len(vals)):
#                 qml.Hadamard(wires = hadamard_wire)

#             # Apply value dependent Z-axis rotations
#             for rotation_val in range(len(vals)):
#                 qml.RZ(vals[rotation_val], wires = rotation_val)

#             # Random CNOT
#             qml.CNOT(wires = [1,0])

#             # Parametized rotation <- this is what is being trained
#             qml.RY(theta, wires=0)

#             # Get expected values & return them
#             expected_values = [qml.expval(qml.PauliX(wire)) for wire in range(len(vals))]
#             return expected_values


#         @qml.qnode(self.devY)
#         def rotation_circuitY(vals, theta):

#             # Apply Hadamards
#             for hadamard_wire in range(len(vals)):
#                 qml.Hadamard(wires = hadamard_wire)

#             # Apply value dependent Z-axis rotations
#             for rotation_val in range(len(vals)):
#                 qml.RZ(vals[rotation_val], wires = rotation_val)

#             # Random CNOT
#             qml.CNOT(wires = [1,0])

#             # Parametized rotation <- this is what is being trained
#             qml.RY(theta, wires=0)

#             # Get expected values & return them
#             expected_values = [qml.expval(qml.PauliX(wire)) for wire in range(len(vals))]
#             return expected_values


#         @qml.qnode(self.devZ)
#         def rotation_circuitZ(vals, theta):

#             # Apply Hadamards
#             for hadamard_wire in range(len(vals)):
#                 qml.Hadamard(wires = hadamard_wire)

#             # Apply value dependent Z-axis rotations
#             for rotation_val in range(len(vals)):
#                 qml.RZ(vals[rotation_val], wires = rotation_val)

#             # Random CNOT
#             qml.CNOT(wires = [1,0])

#             # Parametized rotation <- this is what is being trained
#             qml.RY(theta, wires=0)

#             # Get expected values & return them
#             expected_values = [qml.expval(qml.PauliX(wire)) for wire in range(len(vals))]
#             return expected_values


In [303]:
devX = qml.device('default.qubit', wires = n_vals)
devY = qml.device('default.qubit', wires = n_vals)
devZ = qml.device('default.qubit', wires = n_vals)

@qml.qnode(devX)
def rotation_circuitX(vals, thetas):

    # Apply Hadamards
    for hadamard_wire in range(len(vals)):
        qml.Hadamard(wires = hadamard_wire)

    # Apply value dependent Z-axis rotations
    for rotation_val in range(len(vals)):
        qml.RZ(vals[rotation_val], wires = rotation_val)

    # Random CNOT
    qml.CNOT(wires = [1,0])

    # Parametized rotation <- this is what is being trained
    for theta in range(len(thetas)):
        qml.RY(thetas[theta], wires=theta)

    # Get expected values & return them
    expected_values = [qml.expval(qml.PauliX(wire)) for wire in range(len(vals))]
    return expected_values


@qml.qnode(devY)
def rotation_circuitY(vals, thetas):

    # Apply Hadamards
    for hadamard_wire in range(len(vals)):
        qml.Hadamard(wires = hadamard_wire)

    # Apply value dependent Z-axis rotations
    for rotation_val in range(len(vals)):
        qml.RZ(vals[rotation_val], wires = rotation_val)

    # Random CNOT
    qml.CNOT(wires = [1,0])

    # Parametized rotation <- this is what is being trained
    for theta in range(len(thetas)):
        qml.RY(thetas[theta], wires=theta)

    # Get expected values & return them
    expected_values = [qml.expval(qml.PauliY(wire)) for wire in range(len(vals))]
    return expected_values


@qml.qnode(devZ)
def rotation_circuitZ(vals, thetas):

    # Apply Hadamards
    for hadamard_wire in range(len(vals)):
        qml.Hadamard(wires = hadamard_wire)

    # Apply value dependent Z-axis rotations
    for rotation_val in range(len(vals)):
        qml.RZ(vals[rotation_val], wires = rotation_val)

    # Random CNOT
    qml.CNOT(wires = [1,0])

    # Parametized rotation <- this is what is being trained
    for theta in range(len(thetas)):
        qml.RY(thetas[theta], wires=theta)

    # Get expected values & return them
    expected_values = [qml.expval(qml.PauliZ(wire)) for wire in range(len(vals))]
    return expected_values


In [304]:
rotation_circuitX(x, theta)

ValueError: cannot reshape array of size 10 into shape (2,)

In [305]:
QuantumLayer(4).rotation_circuit([.1, .2, .3, .4], .4)

AttributeError: 'QuantumLayer' object has no attribute 'rotation_circuit'

In [424]:
###### Define our model
class QuantumCicuitNet(torch.nn.Module):
    def __init__(self, n_vals, n_dim):
        super().__init__()
        # self.thetas = torch.nn.Parameter(torch.tensor(np.random.rand(num_rotations)*.0001, dtype = torch.float32, requires_grad=True))         
        # self.QCLX = devX(n_vals)
        # self.QCLY = devY(n_vals)
        # self.QCLZ = devZ(n_vals)
        self.relu = torch.nn.ReLU(inplace=True)
        # self.linear = torch.nn.Linear(n_vals * n_dim, n_dim)
        self.conv2d = torch.nn.Conv2d(1, 1, kernel_size=(3,5), stride=(2,2), padding=(1,1), bias=False)
        self.linear = torch.nn.Linear(4, 1)  


    def param_shift(self, vals, thetas, axis):
        # using the convention u=1/2
        # print("Axis: ", axis)
        if axis == 'X':
            r_plus = rotation_circuitX(vals, np.array(theta) + np.array(np.pi / 2))
            r_minus = rotation_circuitX(vals, np.array(theta) - np.array(np.pi / 2))
            return 10 * (r_plus - r_minus)
        elif axis == 'Y':
            r_plus = rotation_circuitY(vals, np.array(theta) + np.array(np.pi / 2))
            r_minus = rotation_circuitY(vals, np.array(theta) - np.array(np.pi / 2))
            return 10 * (r_plus - r_minus)
        elif axis == 'Z':
            r_plus = rotation_circuitZ(vals, np.array(theta) + np.array(np.pi / 2))
            r_minus = rotation_circuitZ(vals, np.array(theta) - np.array(np.pi / 2))
            return .5 * (r_plus - r_minus)
            
        # print("R Minus: ", r_minus)
        # print("Minus: ", (r_plus - r_minus))
        

    def calc_circ_grad(self, x, theta, axis):
        param_shift_vals = self.param_shift(x, theta, axis)
        # print("Gradient: ", param_shift_vals)
        return param_shift_vals

        
    def forward(self, x, thetas):
        outX = torch.tensor(rotation_circuitX(x, thetas), dtype = torch.float32) * 100# OUT:  torch.Size([100, 1, 10, 10])
        outY = torch.tensor(rotation_circuitY(x, thetas), dtype = torch.float32) * 100# OUT:  torch.Size([100, 1, 10, 10])
        outZ = torch.tensor(rotation_circuitZ(x, thetas), dtype = torch.float32) * 100# OUT:  torch.Size([100, 1, 10, 10])
        out = torch.reshape(torch.cat((outX, outY,outZ), 0), (1, 1, 3, 5))
        # print(out)
        # print("X: ", out)
        # print("Y: ", outY)
        # print("Z: ", outZ)
        # print()
        # exp_vals = deepcopy(out)
        gradX = torch.tensor(self.calc_circ_grad(x, thetas, 'X'))
        gradY = torch.tensor(self.calc_circ_grad(x, thetas, 'Y'))
        gradZ = torch.tensor(self.calc_circ_grad(x, thetas, 'Z'))
        grads = torch.reshape(torch.cat((gradX, gradY,gradZ), 0), (3,5))
        # print(grads)
        grads = torch.mean(input = grads, dim = 0)
        # print("Grads: ", torch.mean(input = grads, dim = 0))
        # print("GradY: ", gradY)
        # print("GradZ: ", gradZ)
        # torch.mean()
        out = self.conv2d(out)
        out = self.relu(out)
        out = out.flatten()
        out = self.linear(out)
        return out, grads

In [425]:
from sklearn import preprocessing
devSet = pd.read_csv("./us_migration.csv")
devSet = devSet.loc[:, ~devSet.columns.str.contains('^Unnamed')]
devSet = devSet.apply(lambda x: pd.to_numeric(x, errors='coerce'))
devSet = devSet.dropna(axis=1)

y = torch.Tensor(devSet['US_MIG_05_10'].values)
X = devSet.loc[:, devSet.columns != "US_MIG_05_10"].values

mMScale = preprocessing.MinMaxScaler()
X = mMScale.fit_transform(X)

x = np.reshape(X[0][0:5], (1, 5))
y = torch.tensor(y.detach().numpy()[0])

print(x)
print(y)


[[0.2898364  0.15086376 0.01904088 0.17205208 0.13949271]]
tensor(961.)


In [440]:
lr = 1e-7
model = QuantumCicuitNet(5, 1)
thetas = [.5] * 5

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)




for i in range(0, 500):

    print("EPOCH: ", i)

    y_pred, grad = model([0.2898364, 0.15086376, 0.01904088, 0.17205208, 0.13949271], thetas)

    loss = criterion(y_pred, y)

    print("    Loss: ", loss)

    print("    Y pred: ", y_pred)
    # print("Gradient: ", gradient[0])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    thetas = [i.item() for i in list(torch.tensor(thetas) + (grad * lr))]

    # print("New theta: ", [i.item() for i in list(torch.tensor(thetas) - (grad * lr))])

    print("\n")

ard>)
    Y pred:  tensor([955.7786], grad_fn=<AddBackward0>)


EPOCH:  338
    Loss:  tensor(24.7267, grad_fn=<MseLossBackward>)
    Y pred:  tensor([956.0274], grad_fn=<AddBackward0>)


EPOCH:  339
    Loss:  tensor(22.4276, grad_fn=<MseLossBackward>)
    Y pred:  tensor([956.2642], grad_fn=<AddBackward0>)


EPOCH:  340
    Loss:  tensor(20.3418, grad_fn=<MseLossBackward>)
    Y pred:  tensor([956.4898], grad_fn=<AddBackward0>)


EPOCH:  341
    Loss:  tensor(18.4490, grad_fn=<MseLossBackward>)
    Y pred:  tensor([956.7048], grad_fn=<AddBackward0>)


EPOCH:  342
    Loss:  tensor(16.7333, grad_fn=<MseLossBackward>)
    Y pred:  tensor([956.9094], grad_fn=<AddBackward0>)


EPOCH:  343
    Loss:  tensor(15.1759, grad_fn=<MseLossBackward>)
    Y pred:  tensor([957.1044], grad_fn=<AddBackward0>)


EPOCH:  344
    Loss:  tensor(13.7634, grad_fn=<MseLossBackward>)
    Y pred:  tensor([957.2901], grad_fn=<AddBackward0>)


EPOCH:  345
    Loss:  tensor(12.4831, grad_fn=<MseLossBackward>)
  

In [432]:
for n, p in model.named_parameters():
    print(n)

conv2d.weight
linear.weight
linear.bias


In [320]:
np.array([.5] * 5) + np.array([np.pi/2])

array([2.07079633, 2.07079633, 2.07079633, 2.07079633, 2.07079633])

In [244]:
conv2d = torch.nn.Conv2d(1, 1, kernel_size=(3,5), stride=(2,2), padding=(1,1), bias=False)
hm = conv2d(torch.tensor([[[[ 8.4098e+01,  9.4741e+01,  9.9982e+01,  9.8524e+01,  9.9029e+01],
        [ 0.0000e+00,  1.4402e+01,  1.9040e+00,  1.7120e+01,  1.3904e+01],
        [-4.5943e+01,  0.0000e+00,  0.0000e+00, -5.5511e-15,  0.0000e+00]]]]))
hm

tensor([[[[ -0.1390, -23.0678],
          [ -7.6440,  -7.7049]]]], grad_fn=<ThnnConv2DBackward>)

In [246]:
linear = torch.nn.Linear(4, 1)  

linear(hm.flatten())

tensor([3.2217], grad_fn=<AddBackward0>)

In [132]:
gradient = qml.grad(rotation_circuit, argnum=0)
# grad_vals = [gradient(theta) for theta in angles]
expvals = [rotation_circuit([.14, .80], theta) for theta in angles]
param_shift_vals = [param_shift([.14, .80], theta) for theta in angles]



NameError: name 'angles' is not defined